<a href="https://colab.research.google.com/github/tohooni/DeepRunning/blob/master/Keras_Sequential_%EB%AA%A8%EB%8D%B8_20191229.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 새 섹션

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.set_printoptions(precision=4)



In [ ]:
print(tf.__version__)

2.1.0-rc1


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([8,3,0,2,1])
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [ ]:
for elem in dataset:
  print(elem.numpy())

8
3
0
2
1


In [ ]:
it = iter(dataset)
print(next(it).numpy())

8


In [ ]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4,10]))
dataset1.element_spec

TensorSpec(shape=(10,), dtype=tf.float32, name=None)

In [ ]:
for i in dataset1:
  print(i.numpy())

[0.3751 0.0038 0.634  0.6964 0.857  0.3492 0.6186 0.6393 0.0749 0.767 ]
[0.6022 0.1113 0.9422 0.3945 0.1571 0.7629 0.4427 0.1974 0.5108 0.776 ]
[0.8144 0.3236 0.2899 0.6351 0.7961 0.7682 0.4518 0.665  0.2302 0.0748]
[0.2114 0.2709 0.0729 0.8167 0.6331 0.9601 0.9834 0.6618 0.1924 0.5343]


In [ ]:
dataset2 = tf.data.Dataset.from_tensor_slices((tf.random.uniform([4]),tf.random.uniform([4,100], maxval=100, dtype=tf.int32)))
dataset2.element_spec

(TensorSpec(shape=(), dtype=tf.float32, name=None),
 TensorSpec(shape=(100,), dtype=tf.int32, name=None))

In [ ]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
dataset3.element_spec

(TensorSpec(shape=(10,), dtype=tf.float32, name=None),
 (TensorSpec(shape=(), dtype=tf.float32, name=None),
  TensorSpec(shape=(100,), dtype=tf.int32, name=None)))

In [ ]:
dataset4 = tf.data.Dataset.from_tensors(tf.SparseTensor(indices=[[0,0], [1,2]], values=[1,2], dense_shape=[3,4]))
dataset4.element_spec

SparseTensorSpec(TensorShape([3, 4]), tf.int32)

In [ ]:
dataset4.element_spec.value_type

tensorflow.python.framework.sparse_tensor.SparseTensor

In [ ]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
images, labels = train
images = images/255

dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset

<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

In [ ]:
def count(stop):
  i = 0
  while i < stop:
    yield i
    i += 1

for n in count(5):
  print(n)    

In [ ]:
ds_counter = tf.data.Dataset.from_generator(count, args=[25], output_types = tf.int32, output_shapes =(),)

In [ ]:
ds_counter.element_spec

TensorSpec(shape=(), dtype=tf.int32, name=None)

In [ ]:
for i in ds_counter:
  print(i.numpy())

In [ ]:
for j in ds_counter.repeat().batch(5).take(15):
  print(j.numpy())

In [ ]:
def gen_series():
  i = 0
  while True:
    size = np.random.randint(0,10)
    yield i, np.random.normal(size=(size,))
    i +=1

In [ ]:
for i, series in gen_series():
  print(i, ":", str(series))
  if i > 5:
    break

0 : [0.844]
1 : [ 0.0856 -0.3816 -1.0786  0.6166  1.3466  1.2473  1.6719]
2 : [ 0.175  -0.7953  0.348   0.9691  0.3692  1.0054]
3 : [ 0.589  -0.981  -1.6912 -1.1769 -0.5577]
4 : [-0.777  -0.295  -2.0142 -0.3963  2.3134 -0.8186 -2.2476]
5 : [ 2.1201 -0.0295  1.6152  0.1433  0.2609 -1.7505 -0.6457]
6 : []


In [ ]:
ds_series = tf.data.Dataset.from_generator(
    gen_series,
    output_types=(tf.int32, tf.float32),
    output_shapes=((), (None,))
)
ds_series

<FlatMapDataset shapes: ((), (None,)), types: (tf.int32, tf.float32)>

In [ ]:
ds_series_batch = ds_series.shuffle(20).padded_batch(10, padded_shapes=([], [None]))

ids, sequence_batch = next(iter(ds_series_batch))
print(ids.numpy())
print()
print(sequence_batch.numpy())

[ 1  5 14  6  2 19 24  0 26 21]

[[ 0.2048  0.9074 -0.3861 -0.6207 -0.5749  0.      0.    ]
 [ 0.9343  0.1563 -1.4227  0.      0.      0.      0.    ]
 [ 1.4304  0.7108  0.      0.      0.      0.      0.    ]
 [ 1.8344  0.      0.      0.      0.      0.      0.    ]
 [ 0.453  -0.4432 -0.6672  0.9182  1.172  -1.22   -1.394 ]
 [-0.319  -0.3985 -0.098  -0.9147  0.      0.      0.    ]
 [-0.4525  1.2467 -0.1798 -0.1824 -0.8532  0.      0.    ]
 [ 0.3542  0.6701  1.8363  0.      0.      0.      0.    ]
 [ 1.0357 -1.1224 -0.6613  0.5366  0.      0.      0.    ]
 [ 0.2956 -2.3237 -0.1355  0.      0.      0.      0.    ]]


In [ ]:
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True
)

228818944/228813984 [==============================] - 7s 0us/step


In [ ]:
flowers

'/root/.keras/datasets/flower_photos'

In [ ]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)

In [ ]:
images, labels = next(img_gen.flow_from_directory(flowers))

Found 3670 images belonging to 5 classes.


In [ ]:
images[1]

In [ ]:
fsns_test_file = tf.keras.utils.get_file("fsns.tfrec", "https://storage.googleapis.com/download.tensorflow.org/data/fsns-20160927/testdata/fsns-00000-of-00001")

7905280/7904079 [==============================] - 1s 0us/step


In [ ]:
fsns_test_file

'/root/.keras/datasets/fsns.tfrec'

In [ ]:
dataset = tf.data.TFRecordDataset(filenames=[fsns_test_file])
dataset

TensorSpec(shape=(), dtype=tf.string, name=None)

In [ ]:
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())

parsed.features.feature['image/orig_width']

int64_list {
  value: 600
}

In [ ]:
for i in dataset:

  aa += 0
  print(tf.train.Example.FromString(raw_example.numpy()))
  if aa == 3:
    break

features {
  feature {
    key: "image/class"
    value {
      int64_list {
        value: 49
        value: 11
        value: 5
        value: 0
        value: 47
        value: 5
        value: 21
        value: 21
        value: 5
        value: 33
        value: 12
        value: 7
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
        value: 133
      }
    }
  }
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\211PNG\r\n\032\n\000\000\000\rIHDR\000\000\002X\000\000\000\226\010\002\000\000\000\215yaf\000\000\000\tpHYs\000\000\000\000\000\000\0